In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import string
import operator
from numpy import nan
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Dropout
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import os
import pickle
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [2]:
data=pd.read_csv('../input/youtube-video-titles/YT.csv')

In [3]:
data=data[['title','video_id','categoryId']]
print(data.shape)
data.head()


(111511, 3)


,title,video_id,categoryId
0,i asked her to be my girlfriend,3C66w5Z0ixs,22
1,apex legends stories from the outlands the end...,M9Pmf9AB4Mo,20
2,i left youtube for a month and this is what ha...,J78aPJ3VyNs,24
3,xxl freshman class revealed official announcement,kXLn3HkpjaA,10
4,ultimate diy home movie theater for the labran...,VIUo6yapDbc,26


In [4]:
title=data['title']
titles=[]
words=set()
count=0
max_length=0
s=''
for i,l in enumerate(title):
    max_length=max(max_length,len(l))
    if type(l)!=str:
        title[i]='x'
    for word in l.split():
        titles.append(word)
        if word not in words:
            words.add(word)
words=list(words)

In [5]:
print("total number of uniq words in dataset are ",len(words))
print("total words in words in the dataset are ",len(titles))
print("maximum length of title is  ",max_length)

total number of uniq words in dataset are  60431
total words in words in the dataset are  916749
maximum length of title is   100


In [6]:
data['categoryId'].value_counts()

24    35021
10    12702
22    10168
17     9728
25     7440
23     6792
20     6574
27     5705
26     5630
28     4825
1      3098
19     1872
2      1159
15      487
29      164
43      144
30        2
Name: categoryId, dtype: int64

In [7]:
x_train,x_test,y_train,y_test=train_test_split(data['title'],data['categoryId'],test_size=.1)
print(len(x_train),len(x_test),len(y_train),len(y_test))

100359 11152 100359 11152


In [8]:
tokenizer=Tokenizer(
    num_words=len(words),
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True, split=' ', char_level=False,
    document_count=0,
)

tokenizer.fit_on_texts(data['title'])
wordIndex=tokenizer.word_index

In [9]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
print(wordIndex['the'])
print(wordIndex['official'])

1
9


In [11]:
x_train=tokenizer.texts_to_sequences(x_train)
print(x_train[:10])

[[1307, 1989, 4335, 2798, 105, 70, 12, 725, 2299, 850], [302, 117, 1, 10410], [16, 22, 5366, 3, 503, 93, 90, 44, 99, 101, 68], [1778, 89, 17, 6, 150, 58], [3832, 3553, 712, 3748], [1709, 59496, 10, 1025, 1962, 1510, 32055], [929, 1211, 1, 135, 1785, 11167, 3, 126], [1990, 1991, 2433, 1533, 3132, 4], [125, 23, 703, 2, 376, 635, 2759, 164, 11443], [1, 8020, 7786, 160, 27, 1581, 1876, 7, 794, 2107]]


In [12]:
x_train=pad_sequences(x_train,maxlen=max_length)
print(x_train[:2])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0  1307  1989  4335  2798   105    70
     12   725  2299   850]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0    

In [13]:
x_test=tokenizer.texts_to_sequences(x_test)
print(x_test[:10])

[[1516, 29529, 3772, 1641, 3124, 110, 13002, 3690, 5608, 70], [27750, 2167, 3307, 511], [1, 8655, 17882, 5725, 3369, 4658, 780], [1565, 5, 4239, 1351, 140, 12, 4239, 1351, 6792], [973, 82, 6013, 286, 59, 261, 20496, 10584, 72, 321, 479, 33], [21806, 21807, 21808, 6, 1217], [11710, 15, 6605, 60, 1520, 2682, 52, 724, 1298, 11712, 6605, 1776, 370, 895, 316, 235], [393, 6728, 44415, 2296, 42, 8, 2296], [186, 336, 15123, 6755, 2009, 48], [406, 154, 571, 629, 122, 35, 86, 25]]


In [14]:
x_test=pad_sequences(x_test,maxlen=max_length)
print(x_test[:2])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0  1516 29529  3772  1641  3124   110
  13002  3690  5608    70]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0    

In [15]:
model=Sequential()
model.add(Embedding(len(words),100,input_length=max_length))
model.add(Dropout(0.2))
model.add(LSTM(100,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(44,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

2022-01-04 16:25:57.666712: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 16:25:57.765173: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 16:25:57.765930: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-01-04 16:25:57.767353: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          6043100   
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 44)                4444      
Total params: 6,127,944
Trainable params: 6,127,944
Non-trainable params: 0
_________________________________________________________________


In [16]:
size=64
epochs=20

In [17]:
y_train = np.array(y_train)
history=model.fit(x_train,y_train,
                    epochs=epochs,
                    validation_data =(x_test,y_test))
# pickle.dump(history, open("history.p", "wb"))
model.save_weights('YTlstm.h5')

2022-01-04 16:26:00.437907: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
3137/3137 [==============================] - 1362s 433ms/step - loss: 1.3266 - accuracy: 0.5983 - val_loss: 0.9518 - val_accuracy: 0.7036
Epoch 2/20
3137/3137 [==============================] - 1405s 448ms/step - loss: 0.7284 - accuracy: 0.7742 - val_loss: 0.8678 - val_accuracy: 0.7298
Epoch 3/20
3137/3137 [==============================] - 1445s 461ms/step - loss: 0.5165 - accuracy: 0.8374 - val_loss: 0.9186 - val_accuracy: 0.7283
Epoch 4/20
3137/3137 [==============================] - 1416s 451ms/step - loss: 0.3982 - accuracy: 0.8729 - val_loss: 0.9912 - val_accuracy: 0.7222
Epoch 5/20
3137/3137 [==============================] - 1434s 457ms/step - loss: 0.3236 - accuracy: 0.8949 - val_loss: 1.0606 - val_accuracy: 0.7203
Epoch 6/20
3137/3137 [==============================] - 1442s 460ms/step - loss: 0.2754 - accuracy: 0.9092 - val_loss: 1.1354 - val_accuracy: 0.7162
Epoch 7/20
3137/3137 [==============================] - 1408s 449ms/step - loss: 0.2394 - accuracy: 0.9206

In [18]:
# model.load_weights("../input/load-models/YTlstm.h5")

In [19]:
scores = model.evaluate(x_test, y_test, verbose=0)
print(scores)

[1.9516841173171997, 0.7068687081336975]


In [20]:
# a=[x_test[0]]


In [21]:
# y_pre = model.predict(x_test)

In [22]:
# y_pred = np.argmax(y_pre, axis=1)

In [23]:
# print(y_pred)

In [24]:
# print(y_test)

In [25]:
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

TypeError: 'History' object is not subscriptable

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')